# Задание 1

In [6]:
import requests

response = requests.get("https://jsonplaceholder.typicode.com/posts")
data = response.json()
for i in range(0, 5):
    print(data[i]["title"], end = "\n")


sunt aut facere repellat provident occaecati excepturi optio reprehenderit
qui est esse
ea molestias quasi exercitationem repellat qui ipsa sit aut
eum et est occaecati
nesciunt quas odio


# Задание 2

In [7]:
city = input()
city

'London'

In [ ]:
api_key = ''

In [ ]:
url = f'http://api.openweathermap.org/geo/1.0/direct?q=London&appid={api_key}'
url

In [ ]:
city_response = requests.get(url)
print(city_response)